# Aggregation (spatial downsampling)

### Continuous aggregation

The core aggregation code is written in Cython, in `raster_utilities.aggregation.spatial.core.continuous.pyx`. 

A helper class `raster_utilities.aggregation.spatial.SpatialAggregator` is provided to manage calling the Cython code.

This notebook demonstrates using the helper class to aggregate a series of continuous-type raster files (i.e. Float32 values where we want to summarise mean, max, min etc rather than mode, percentage etc in the case of categorical data).

The code has been written to read input rasters of theoretically unlimited size, which are read in tiles to build up the output coarser / smaller grids; memory use is determined by the size of the output files (and the number of statistics requested, i.e. number of output files that are created). 

Note that there is no specific need for the input data to be a .tif. It can be any GDAL-compatible format so long as the data are single-band and 32-bit float type. It has been used to aggregate a global 7-metre resolution grid to mastergrids 1k, reading from a .VRT file to avoid the need to ever generate the mosaiced high-resolution grid as a (huge) tiff file.

In [1]:
# The helper class
from  raster_utilities.aggregation.spatial.SpatialAggregator import *

In [2]:
# Enumerations to provide acceptable values for the aggregation parameters,
# avoid having to remember strings
from raster_utilities.aggregation.aggregation_values import *

In [3]:
import glob

### Run a continuous aggregation across a series of files in a folder

In [22]:
# The files to be aggregated should be provided as a list of filepaths. 
# (Just make a single-item list for one file)
#inContFiles = glob.glob(r'H:\Night\1km\Monthly\LST_Night_v6.*.max.*tif')
inContFiles = glob.glob(r'\\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\MODIS_Global\MOD11A2_v6_LST\Modelled_Air_Temp_Min\1km\Synoptic\*.tif')
#inContFiles = glob.glob(r'\\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\Other_Global_Covariates\NightTimeLights\DMSP_Intercalibrated_Series\1km\Annual\*.tif')

# Also provide the output folder
#outDir = r'Z:\mastergrids\MODIS_Global\MOD11A2_v6_LST\LST_Night\5km\Monthly'
#outDir = r'E:\Data\Harry\Documents\dial-a-map\andre_pop'
outDir = r'C:\Temp\modis_air_min'

Specify the output nodata value (it doesn't have to be the same as the input, incoming NDV will be read from the files (better be set properly!)

In [23]:
ndvOut = -9999

Specify the aggregation statistics to create. This must be a list of items from the ContinuousAggregationStats enumeration, or their string representations. 

The SpatialAggregator determines whether it is supposed to be doing continuous or categorical aggregation based on which stats are requested, and the data type of the files provided must match.

In [24]:
# e.g.
stats = [ContinuousAggregationStats.MEAN, ContinuousAggregationStats.MAX,
         ContinuousAggregationStats.MIN, ContinuousAggregationStats.SD]
# stats = [ContinuousAggregationStats.SUM]
#stats = [ContinuousAggregationStats.MEAN]
# or do do all of them use this convenience: 
#stats = ContinuousAggregationStats.ALL.value

Finally configure the aggregation. The final parameter for the SpatialAggregator constructor should be a dictionary that configures how the aggregation will run. 

* There should be a key '`aggregation_type`' that is a member of the AggregationTypes enumeration, i.e. `AggregationTypes.RESOLUTION`, `AggregationTypes.FACTOR`, or `AggregationTypes.SIZE`. 
* There should be a key 'aggregation_specifier' that determines the output cell size in a manner dependent on the 
value of aggregation_type as follows:
    * `aggregation_type==AggregationTypes.RESOLUTION`: (Float value, or string "1km", "5km" or "10km")
    * `aggregation_type==AggregationTypes.FACTOR`: Int value (e.g. 5 to go from 1k rasters to 5k rasters
    * `aggregation_type==AggregationTypes.SIZE`: 2-tuple of positive ints specifying the (height,width) of the output rasters
* A key "`resolution_name`" may be provided, which provides the "friendly name" for the output resolution to be used as the fifth token of the 6-token output filenames (e.g. "5km")
* A key "`mem_limit_gb`" may be provided, to limit the memory use (if not provided, 30GB will be the default). Note that it's not very accurate so be conservative!
* A key "`assume_correct_input`" may be provided; if this is "`False`" (by default if not provided) then the input data will be snapped and aligned to a mastergrid template first, before calculating the properties of the output raster
* A key "`sanitise_resolution`" may be provided; if this is "`True`" (by default if not provided) then the output resolution (whether provided numerically or calculated) will be "sanitised" to a mastergrid resolution i.e. a value that divides cleanly into 1.0. For example 0.0083334 would become 0.008333333333333 (1/120)).
* A key "`snap_alignment`" may be provided; if this is `SnapTypes.NEAREST` (the default if not provided) or `SnapTypes.TOWARDS_ORIGIN` then the origin point of the output will be positioned precisely at the top left corner of a cell in a global grid of the requested resolution. Because this potentially moves the extreme (bottom right) point of the output towards the origin such that it is inside the bottom right of the input data, an extra cell will be added if necessary to the output extent to accommodate the full input extent.

In [25]:
# e.g.
# Resolution can be a floating point number, or a string representing 
# one of the core mastergrid resolutions "1km", "5km", or "10km".
aggArgs = {'aggregation_type': AggregationTypes.RESOLUTION
           , 'aggregation_specifier': "5km"
           , 'resolution_name':'5km'
           , 'sanitise_resolution':True
           , 'snap_alignment': SnapTypes.NEAREST
           , 'assume_correct_input': False
           , 'mem_limit_gb': 10
          }

### Running - Single-process

Now just instantiate and run the aggregation, processing one input file at a time:

In [18]:
agg = SpatialAggregator(inContFiles, outDir, ndvOut, stats, aggArgs, LogLevels.DEBUG)


In [ ]:
agg.RunAggregation()

### Running - multiprocessing

Or use multiprocessing to do several files at once - the continuous aggregation algorithm is single-threaded so use multiprocessing instead to make gains. Pick a pool size that corresponds to the number of cores to run at once; keep an eye on disk utilisation as this will become the bottleneck and if it's pegged at 100% then that will end up slower so it'll be better to use fewer processes. (The compression algorithm is multithreaded when saving, but don't really need to worry about that). A pool size of 4 is probably as big as is worthwhile on an average PC especially if the source files are being read over a network or spinning disk.

https://medium.com/@grvsinghal/speed-up-your-python-code-using-multiprocessing-on-windows-and-jupyter-or-ipython-2714b49d6fac

In [10]:
# due to some weird reason this function needs to be defined externally and imported 
#def callAgg(f):
#    try:
#        agg = SpatialAggregator([f], outDir, ndvOut, stats, aggArgs)
#        agg.RunAggregation()
#    except KeyboardInterrupt, e:
#        pass


In [28]:
from multiprocessing import Pool
from spatial_agg_worker import callAgg

# now we can just do this:
#p = Pool(4)
#p.map_async(callAgg, inContFiles)
# but it is impossible to interrupt if we need to! to allow that, need to do this:
    # https://bryceboe.com/2010/08/26/python-multiprocessing-and-keyboardinterrupt/

# as we have to import callAgg it won't have access to the globals outDir etc so zip them with the filename 
# into a single object that can be passed via pool.map
zippedArgs = [(f,outDir,ndvOut,stats,aggArgs) for f in inContFiles]
def runMulti():
    # choose an number not greater than the number of cores, but also that won't use more than 
    # the available memory and preferably substantially less so that OS-level write-caching can 
    # help prevent the disk becoming a bottleneck (ensure you are writing to a disk with write
    # caching enabled: it isn't by default on external drives)
    pool = Pool(4)
    p = pool.map_async(callAgg, zippedArgs)
    try:
        r = p.get(0xFFFF)
    except KeyboardInterrupt:
        print ("parent received interrupt")
        return
    


In [29]:
# call it
runMulti()